# Recurso Pedagógico: Análisis de Calificaciones

## 1. Inicio

In [1]:
"""
analisis_calificaciones_plotly.py
Análisis de calificaciones con Plotly (todo local).
Entrada: DatosCursoIA.xlsx (hoja por defecto)
Salida: plots/*.html y processed_data.csv
"""

import os
import pandas as pd
import numpy as np

# Plots
import plotly.graph_objects as go
import plotly.express as px

# ML & stats
from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
#from sklearn.ensemble import IsolationForest
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from scipy import stats

# ---------------------------
# CONFIG
# ---------------------------

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# **IMPORTANTE**: Reemplaza esta ruta con la ubicación real de tu archivo en Google Drive
INPUT_FILE = "/content/drive/MyDrive/DatosCursoAI.xlsx"   # Ruta actualizada para Google Drive
OUT_DIR = "plots"
PROCESSED_CSV = "processed_data.csv"
os.makedirs(OUT_DIR, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Carga de datos y preparación

In [2]:
# ---------------------------
# CARGA Y PREPARACIÓN
# ---------------------------
df_raw = pd.read_excel(INPUT_FILE)

# Si la primera columna es un índice o 'Periodo' con NaN (como en tu muestra),
# vamos a asumir que las columnas con nombres tipo "2021-2", "2022-1", ... son las de calificaciones.
# Seleccionamos columnas numéricas (o con formato año-semestre).
# Hecho robusto: conservamos columnas que puedan convertirse a float.
def is_numeric_col(s):
    try:
        pd.to_numeric(df_raw[s])
        return True
    except Exception:
        return False

# si hay columnas no numéricas (p.ej. nombres), las removemos
numeric_cols = [c for c in df_raw.columns if is_numeric_col(c)]
if len(numeric_cols) == 0:
    # fallback: todas las columnas excepto la primera
    numeric_cols = list(df_raw.columns[1:])

df = df_raw[numeric_cols].copy()
# Renombrar columnas para orden lógico si están como strings "YYYY-S"
# Mantener el orden original del archivo
periods = list(df.columns)

# Crear IDs anónimos
n_students = df.shape[0]
anon_ids = [f"S{idx+1}" for idx in range(n_students)]
df.index = anon_ids

# Asegurarnos que los datos sean numéricos
df = df.apply(pd.to_numeric, errors='coerce')

# Guardar copia procesada con IDs anónimos
df.to_csv(PROCESSED_CSV, index=True)

3. Visualización del promedio por semestre

In [3]:
# ---------------------------
# MÉTRICAS BÁSICAS
# ---------------------------
means = df.mean(axis=0, skipna=True)
stds = df.std(axis=0, ddof=1, skipna=True)
counts = df.count(axis=0)

# ---------------------------
# Tendencia: promedio por periodo + proyección
# ---------------------------
# Serie de promedios
series = means.rename('mean').to_frame()

# Modelo simple de suavizado exponencial para proyectar 1 periodo adelante
# Requiere al menos 2 puntos; se captura excepción si no es posible.
forecast_steps = 1
projection = None
try:
    model = ExponentialSmoothing(series['mean'], trend="add", seasonal=None, initialization_method="estimated")
    fit = model.fit(optimized=True)
    fcast = fit.forecast(forecast_steps)
    projection = fcast.iloc[0]
except Exception as e:
    projection = None

# Gráfico lineal con tendencia (line) y punto proyectado
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=series.index, y=series['mean'],
                               mode='lines+markers', name='Promedio por periodo'))
# línea de tendencia lineal (regresión simple)
x_num = np.arange(len(series))
coef = np.polyfit(x_num, series['mean'].values, 1)
trend_line = np.poly1d(coef)(x_num)
fig_trend.add_trace(go.Scatter(x=series.index, y=trend_line,
                               mode='lines', name='Tendencia lineal', line=dict(dash='dash')))
if projection is not None:
    next_label = f"{series.index[-1]} → next"
    fig_trend.add_trace(go.Scatter(x=[next_label], y=[projection],
                                   mode='markers', name='Proyección (1 periodo)',
                                   marker=dict(symbol='diamond', size=12)))
fig_trend.update_layout(title="Promedio del curso por periodo y proyección (1 periodo)",
                        xaxis_title="Periodo", yaxis_title="Promedio")
fig_trend.write_html(os.path.join(OUT_DIR, "tendencia_promedio.html"))
fig_trend.show()

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


## 4. Visualización de la dispersión

In [4]:
# ---------------------------
# Dispersión: boxplots por periodo (ver variabilidad)
# ---------------------------
fig_box = go.Figure()
for p in periods:
    fig_box.add_trace(go.Box(y=df[p], name=p, boxpoints="outliers"))
fig_box.update_layout(title="Boxplot de calificaciones por periodo",
                      xaxis_title="Periodo", yaxis_title="Calificación")
fig_box.write_html(os.path.join(OUT_DIR, "boxplot_periodos.html"))
fig_box.show()

## 5. Visualización de la Correlación entre periodos

In [5]:
# ---------------------------
# Correlación entre periodos (heatmap)
# ---------------------------
corr = df.corr()
fig_corr = px.imshow(corr, text_auto=True, title="Matriz de correlación entre periodos")
fig_corr.write_html(os.path.join(OUT_DIR, "correlation_heatmap.html"))
fig_corr.show()

## 6. Visualizzación de la frecuencia de las calificaciones en cada periodo

In [6]:
# Seleccionar solo columnas numéricas
columnas = df.select_dtypes(include=["number"]).columns

# Convertir el DataFrame a formato largo para graficar todas las columnas juntas
df_melt = df[columnas].melt(var_name="Variable", value_name="Valor")

fig_hist1 = px.histogram(
    df_melt,
    x="Valor",
    color="Variable",
    nbins=20,             # puedes ajustar el número de bins
    barmode="overlay",    # superpone los histogramas
    opacity=0.5,
    title="Histograma superpuesto de todos los periodos"
)

fig_hist1.update_layout(
    template="simple_white",
    bargap=0.1
)

fig_hist1.write_html(os.path.join(OUT_DIR, "histograma1.html"))
fig_hist1.show()

## 7. Visualización de la frecuencia total de las calificaciones
En este caso se suman las ocurrencias de las calificaciones a lo largo de todos los periodos.

In [8]:
# Seleccionar solo columnas numéricas (los periodos)
columnas = df.select_dtypes(include="number").columns

# Combinar todas las columnas en un solo vector
valores_combinados = pd.concat([df[col] for col in columnas], ignore_index=True)

# Graficar un histograma único
fig_hist2 = px.histogram(
    valores_combinados,
    nbins=20,
    title="Histograma con los counts sumados de todos los periodos",
    labels={"value": "Calificaciones"},
    opacity=0.75
)

fig_hist2.update_layout(template="simple_white")
fig_hist2.write_html(os.path.join(OUT_DIR, "histograma1.html"))
fig_hist2.show()